# Treinamento e Teste das CNNs para classificação de LIBRAS

peguei muita coisa desse site !
https://www.datacamp.com/tutorial/cnn-tensorflow-python

## Importações, declaração de variáveis globais e carregamento dos dados

In [ ]:
# importações iniciais
import os
import cv2
import numpy as np 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
#variáveis globais
INPUT_SHAPE = (64, 64, 1)
FILTER1_SZ = 32
FILTER2_SZ = 64
FILTER_SHAPE = (3, 3)
POOL_SHAPE = (2, 2)
FULLY_CONNECT_NUM = 128
NUM_CLASSES = 21

BATCH_SIZE = 64
EPOCHS = 30

METRICS = metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]

classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y']

img_size=(64, 64)

class_to_idx = {c: i for i, c in enumerate(classes)}

In [ ]:
#função auxiliar para função de carregar os dados
def load_split(split, root):
    imgs = []
    labels = []
    split_dir = os.path.join(root, split)
    for cls in classes:
        cls_dir = os.path.join(split_dir, cls)
        if not os.path.isdir(cls_dir):
            continue
        for fname in os.listdir(cls_dir):
            if not fname.lower().endswith(".png"):
                continue
            path = os.path.join(cls_dir, fname)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            img = img.astype("float32") / 255.0
            img = np.expand_dims(img, axis=-1)  # (H, W, 1)
            imgs.append(img)
            labels.append(class_to_idx[cls])
    imgs = np.stack(imgs, axis=0)
    labels = np.array(labels, dtype="int32")
    return imgs, labels

In [ ]:
#função para carregar os dados

def load_libras_data(raw_root="data_balanced", pre_root="data_balanced_preprocessed", img_size=img_size):
    # dados brutos
    train_imgs, train_labels_int = load_split("train", raw_root)
    test_imgs, test_labels_int = load_split("test", raw_root)

    # dados pré-processados
    train_imgs_pre, _ = load_split("train", pre_root)
    test_imgs_pre, _ = load_split("test", pre_root)
    
    # one-hot para Keras
    num_classes = len(classes)
    train_labels = to_categorical(train_labels_int, num_classes=num_classes)
    test_labels = to_categorical(test_labels_int, num_classes=num_classes)

    return (train_imgs, train_imgs_pre, train_labels,
            test_imgs, test_imgs_pre, test_labels)
    

In [ ]:
# carregando dados
(train_imgs,
 train_imgs_preprocessed,
 train_labels,
 test_imgs,
 test_imgs_preprocessed,
 test_labels) = load_libras_data()


## Implementação da arquitetura do modelo

In [ ]:
def LibrasCNN(INPUT_SHAPE=INPUT_SHAPE, NUM_CLASSES=NUM_CLASSES):
    model = Sequential()
    model.add(Conv2D(FILTER1_SZ, FILTER_SHAPE, activation='relu', input_shape=INPUT_SHAPE))
    model.add(MaxPooling2D(POOL_SHAPE))
    model.add(Conv2D(FILTER2_SZ, FILTER_SHAPE, activation='relu'))
    model.add(MaxPooling2D(POOL_SHAPE))
    model.add(Flatten())
    model.add(Dense(FULLY_CONNECT_NUM, activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    return model

## Sumário

In [ ]:
preprocessed_model = LibrasCNN()
non_preprocessed_model = LibrasCNN()

preprocessed_model.summary()

## Treinamento

### CNN com dados sem pré-processamento em OpenCV

In [ ]:
non_preprocessed_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [ ]:
training_history1 = preprocessed_model.fit(train_imgs, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_imgs, test_labels))

### CNN com dados pré-processados em OpenCV

In [ ]:
preprocessed_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)


In [ ]:
training_history2 = preprocessed_model.fit(train_imgs_preprocessed, train_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_imgs_preprocessed, test_labels))